K-means es es el método comunmente utilizado a la hora de hacer clusstering. Define k centroides, uno por cluster, y los dato son asociados al centroide más cercano.

El proceso de preparación de datos es en la mayoría de pasos el común, con la excepción de que se deben normalizar los datos numéricos entre 0 y 1, y que al no existir una variable numérica no es necesario aplicar un LabelEncoder, ni una división 70-30 de los datos.

Los hiperparámetros del modelo son los siguientes:

- k: Cantidad de clusters
- Función de distancia.
- Iteraciones: Una cantidad de iteraciones máximas a realizar.

Las medidas de evaluación de k-means son:

- Cohesión: Qué tan juntos están los datos con respecto al (los) centroides. Mientras menor sea, mejor.

- Separabilidad: Qué tan distantes están los clussters los unos de otros. Mientras mayor sea, mejor.

En esta plantilla se asume que ya se cuentan con los datos limpios y preparados, sin la aplicación de LabelEncoder. Luego en esta misma plantilla se normalizan las variables numéricas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Normalización de variables numéricas

Dada la presencia de variables numéricas se procede a normalizar.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
numeric_columns
min_max_scaler.fit(df[numeric_columns])

df[numeric_columns]= min_max_scaler.transform(df[numeric_columns])
df.head()

## Aprendizaje

## Método del codo

El método del codo se usa para determinar la cantidad de clusters a usar. Se toma el valor de k para el cual los decrecimientos de inercia empiezan a ser decrecientes.

In [ ]:
from sklearn.cluster import KMeans

ks = range(1, 20) 
inertias = []

for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(df)
    inertias.append(model.inertia_)

plt.plot(ks, inertias, '-o')
plt.xlabel('Numero de clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
k = 7
model = KMeans(n_clusters=k, max_iter=100)
model.fit(df)

In [ ]:
print(model.inertia_)

In [ ]:
centroids = pd.DataFrame(model.cluster_centers_, columns=df.columns)
centroids.round(0)
centroids

Para obtener una salida legible

In [ ]:
centroids[numeric_columns] = min_max_scaler.inverse_transform(centroids[numeric_columns])
centroids.round(0)

Asignando los clusters al DataFrame,

df_clean['cluster'] = model.labels_
df_clean.head()

# Entrenando un modelo para clasificación en base al clustering

Para esto, se harán las siguientes acciones:

1. Dividir el dataset con su cluster asignado con una razón de 70/30.
2. Entrenar un árbol de decisión (o knn) con el 70% de los datos.
3. Dar métricas de la exactitud del modelo.

In [ ]:
from sklearn.tree import TreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Preparación de los datos

In [ ]:
X = df.drop('cluster', axis=1)
y = df['cluster']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

## Entrenamiento del árbol de decisión

In [ ]:
model_Tree = TreeClassifier(criterion='squared_error', min_samples_leaf=2, max_depth=None)
model_Tree.fit(x_train, y_train)

### Gráfica del árbol de decisión

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(25,30))
plot_tree(model_Tree, feature_names=X_train.columns.values,
          class_names=['Presupuesto para invertir'],
          rounded=True, filled=True)
plt.show()

## Entrenamiento del modelo con Knn.

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(x_train, y_train)

## Evaluación de ambos modelos

In [ ]:
y_pred = model_Tree.predict(x_test)
print("accuracy_score del arbol de decisión:", accuracy_score(y_test, y_pred))

In [ ]:
plt.scatter(Y_test, Y_pred)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'k--', lw=2)
plt.xlabel('Valores reales')
plt.ylabel('Valores predichos')
plt.title("Gráfico de dispersión del árbol de decisión")
plt.show()

In [ ]:
y_pred = model_Tree.predict(x_test)
print("accuracy_score del knn:", accuracy_score(y_test, y_pred))

In [ ]:
plt.scatter(Y_test, Y_pred)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'k--', lw=2)
plt.xlabel('Valores reales')
plt.ylabel('Valores predichos')
plt.title("Gráfico de dispersión del árbol de decisión")
plt.show()

## Guardando el modelo

In [ ]:
import pickle
filename = 'model.pkl'
variables= X.columns._values
pickle.dump([model_Tree, model_Knn, variables], open(filename, 'wb'))